In [1]:
import numpy as np
import os

log_dir = 'k_log\ExperimentsKE'
log_list = []
time_list = []
for r, _, files in os.walk(log_dir):
    for f in files:
        fname = os.path.join(r, f)
        if fname.endswith('.log'):
            log_list.append(fname)
        else:
            time_list.append(fname)
# print(time_list)
print(len(log_list))

1001


In [2]:
import re

def parse_time(l):
    pattern = '[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{6}'
    base = os.path.basename(l)
    return re.search(pattern, base).group()

print(log_list[0])
t = parse_time(log_list[0])

print(t)

k_log\ExperimentsKE\log_2021-08-01T222805.log
2021-08-01T222805


In [3]:
def parse_block(block, name, host):
    m = {}
    m['func'] = block[0]
    meter_num = 0
    meter_time = 0
    control =['server', 'supplier', 'center']
    ta = ['ttp', 'kgc', 'ta']
    m['control'] = 0
    m['ta'] = 0
    m['agg'] = 0
    for r in block[2::2]:
        x, y = r.split(':')
        t = sum(int(e) for e in y.split(','))
        if x.startswith('meter_'):
            meter_num += 1
            meter_time += t
        elif x in control:
            m['control'] = +t
        elif x in ta:
            m['ta'] = t
        else:
            m[x] = t
    
    m['meter_num'] = meter_num
    m['meter_time'] = meter_time
    m['name'] = name
    m['host'] = host
    return m

def parse_record(lines):
    k_order = [2,2,2,2,2,2,2,3,4,5,6,7,8,9,10,11,11,11,11,11,12,13,14,15,16,17,18,19,20]
    pattern = '[^ ]*$'
    host_pattern = '[^ ]*'
    name_pattern = 'bjut\.(.*)\.myMain'
    host = re.search(host_pattern, lines[0]).group()
    name = re.findall(name_pattern, lines[0])[0]

    func_block = []
    block = []
    for line in lines:
        s = re.search(pattern, line).group().strip()
        # print(s)
        # parse function block
        if ':' not in s: # enter function
            if len(block) != 0:
                func_block.append(block)
            block = []
            block.append(s)
        else:
            block.append(s)
    if len(block) != 0:
        func_block.append(block)

    # print(func_block[0])
    record_list = []
    for b in func_block:
        r = parse_block(b, name, host)
        record_list.append(r)
    # return record_list
   
    for r, k in zip(record_list[2:-1:3], k_order):
        r['func'] = r['func'] + '_' + str(k)
    return record_list

In [4]:
import csv

linelen_set = {}
for l in log_list:
    with open(l) as fo:
        lines = fo.readlines()
        line_len = len(lines)
        if line_len not in linelen_set:
            linelen_set[line_len] = 1
        else:
            linelen_set[line_len] += 1
print(linelen_set)
right_line_set = []
for k,v in linelen_set.items():
    if v > 20:
        right_line_set.append(k)
print(right_line_set)

files = 0
fields_name = []
with open('k_extend_data.csv', 'w', newline='') as csvfile:
    writer = None
    cur_len = 0
    for l in log_list:
        log_time = parse_time(l)
        with open(l) as fo:
            try:
                lines = fo.readlines()
                line_len = len(lines)
                if line_len not in right_line_set:
                    continue
                record = parse_record(lines)
                if len(fields_name) == 0:
                    fields_name  = fields_name + [k for k in record[0].keys()]
                    fields_name.append('timestamp')
                    writer = csv.DictWriter(csvfile, fieldnames=fields_name)
                    writer.writeheader()
                    print(fields_name)
                for l in record:
                    l['timestamp'] = log_time
                    writer.writerow(l)
            except ValueError as e:
                print(l)
                print(e)
        files += 1

print('sucess')

{3960: 999, 1936: 1, 2024: 1}
[3960]
['func', 'control', 'ta', 'agg', 'meter_num', 'meter_time', 'name', 'host', 'timestamp']
sucess
